In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from pandas.core.common import flatten

(_, _), (x_test, y_test) = keras.datasets.mnist.load_data()
x_test = x_test / 255.0

# Load the trained model from the saved .h5 file
model = keras.models.load_model('NewOptimizedModel.h5')

# Make predictions on the testing data
predictions = model.predict(x_test)

# Convert the predicted probabilities to class labels
predicted_labels = np.argmax(predictions, axis=1)

# Calculate accuracy
accuracy = np.mean(predicted_labels == y_test)

print("Accuracy:", accuracy)

313/313 [==============================] - 1s 3ms/step
Accuracy: 0.098


In [13]:
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers , models

In [40]:
mnist = tf.keras.datasets.mnist
(trainImages , trainLabel) , (testImages , testLabel) = mnist.load_data()
trainImages = trainImages/255.0
testImages  = testImages/255.0


trainImages = trainImages.reshape( -1 , 28 , 28 , 1)
testImages = testImages.reshape(-1, 28 , 28 , 1)
print(testImages.shape)
print(trainImages.shape)

(10000, 28, 28, 1)
(60000, 28, 28, 1)


In [15]:
import random
def generateCentrallisedWeight(weightArr):
    ans = np.zeros(len(weightArr))
    stdDeviation = np.std(weightArr)
    MeanVal = np.average(weightArr)
    for i in range(len(weightArr)):
        ans[i] = random.uniform(MeanVal - stdDeviation , MeanVal + stdDeviation)
    return ans

In [8]:
def changeWeights(arr , size):
    ans  = []
    for i in range(size):
        ans.append(generateCentrallisedWeight(arr[i]))
    return ans

In [25]:
optimizedModel = models.Sequential()
optimizedModel.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
optimizedModel.add(layers.MaxPooling2D((2, 2)))
optimizedModel.add(layers.Conv2D(64, (3, 3), activation='relu'))
optimizedModel.add(layers.MaxPooling2D((2, 2)))
optimizedModel.add(layers.Flatten())
optimizedModel.add(layers.Dense(64, activation='relu'))
optimizedModel.add(layers.Dense(10, activation='softmax'))

size =  optimizedModel.layers[5].weights[0].shape[0]
optimizedWeights = changeWeights(optimizedModel.layers[5].weights[0] , size)
optimizedModel.layers[5].weights[0] = optimizedWeights

size =  optimizedModel.layers[6].weights[0].shape[0]
optimizedWeights = changeWeights(optimizedModel.layers[6].weights[0] , size)
optimizedModel.layers[6].weights[0] = optimizedWeights

optimizedModel.compile(optimizer='adam' , loss  = 'sparse_categorical_crossentropy',  metrics = ['accuracy'])
optimizedModel.fit(trainImages, trainLabel, epochs=1, batch_size=32, validation_data=(testImages, testLabel))
y_predict = optimizedModel.predict(testImages)
pred_list = []
for lisstt in range(len(y_predict)):
    maxelem = -1
    index = 0
    for j  in range(len(y_predict[lisstt])):
        if(y_predict[lisstt][j] >  maxelem):
            index = j
            maxelem = y_predict[lisstt][j]
    pred_list.append(index)
print(pred_list)
print(testLabel)

313/313 [==============================] - 2s 6ms/step
[7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 8, 4, 9, 6, 6, 5, 4, 0, 7, 4, 0, 1, 3, 1, 3, 4, 7, 2, 7, 1, 2, 1, 1, 7, 4, 2, 3, 5, 1, 2, 4, 4, 6, 3, 5, 5, 6, 0, 4, 1, 9, 5, 7, 8, 9, 3, 7, 4, 6, 4, 3, 0, 7, 0, 2, 9, 1, 7, 3, 2, 9, 7, 7, 6, 2, 7, 8, 4, 7, 3, 6, 1, 3, 6, 9, 3, 1, 4, 1, 7, 6, 9, 6, 0, 5, 4, 9, 9, 2, 1, 9, 4, 8, 7, 3, 9, 7, 4, 4, 4, 9, 2, 5, 4, 7, 6, 7, 9, 0, 5, 8, 5, 6, 6, 5, 7, 8, 1, 0, 1, 6, 4, 6, 7, 3, 1, 7, 1, 8, 2, 0, 2, 9, 9, 5, 5, 1, 5, 6, 0, 3, 4, 4, 6, 5, 4, 6, 5, 4, 5, 1, 4, 4, 7, 2, 3, 2, 7, 1, 8, 1, 8, 1, 8, 5, 0, 8, 9, 2, 5, 0, 1, 1, 1, 0, 9, 0, 3, 1, 6, 4, 2, 3, 6, 1, 1, 1, 3, 9, 5, 2, 9, 4, 5, 9, 3, 9, 0, 3, 6, 5, 5, 7, 2, 2, 7, 1, 2, 8, 4, 1, 7, 3, 3, 8, 8, 7, 9, 2, 2, 4, 1, 5, 9, 8, 7, 2, 3, 0, 4, 4, 2, 4, 1, 9, 5, 7, 7, 2, 8, 2, 0, 8, 5, 7, 7, 9, 1, 8, 1, 8, 0, 3, 0, 1, 9, 9, 4, 1, 8, 2, 1, 2, 9, 7, 5, 9, 2, 6, 4, 1, 5, 8, 2, 9, 2, 0, 4, 0, 0, 2, 8, 4, 7, 1, 2, 4, 0, 2, 7, 4, 3, 3, 0, 0, 3, 1,

In [43]:
from matplotlib import pyplot as plt
pred_list = np.array(pred_list)
sample = np.zeros(10000)
for i in range(10000):
    sample[i] = i + 1
count = 0
for i in range(1000):
    if(pred_list[i] == testLabel[i]) :
        count =  count + 1
print((count/10000)  * 100)
for i in range(len)
plt.plot(sample, pred_list)
# plt.plot(sample , testLabel)
plt.show()

IndexError: index 50 is out of bounds for axis 0 with size 50

In [37]:
len(trainImages)

60000